In [1]:
import os
import geoutils as gu

#from helpers import getV, getMH, make_affine_matrix, yield_map
from helpers import getV, getMH, make_affine_matrix, yield_map, gen_heightmap, thicken, split_layer, tesselate_layer ,gen_fill, package_commands

# Parameters & Settings

In [2]:
# Loading rasters

fns = [fn for fn in os.listdir("data") if fn.endswith(".tif")]
rasters = sorted([gu.Raster(os.path.join("data", fn)) for fn in fns], key=lambda r: r.bounds[:2])

# Defining location of 2 points with precisely known longitudes and latitudes in both settings
x0, z0 =  -170, -196
x1, z1 = -1144,  284

w0, h0 = 2533405,   1152605.5
w1, h1 = 2532702.5, 1152259.5

# Creating XZ transformation matrices (both ways)

mX, hX = getMH(x0, w0, x1, w1)
mZ, hZ = getMH(z0, h0, z1, h1)
m2c = make_affine_matrix([mX, mZ], [hX, hZ])

mX, hX = getMH(w0, x0, w1, x1)
mZ, hZ = getMH(h0, z0, h1, z1)
c2m = make_affine_matrix([mX, mZ], [hX, hZ])

# Getting real-world altitude of 2 points with precisely known altitudes in both settings
lonField, latField = 2534190, 1152375
lonEspla, latEspla = 2533030, 1152475

v0 = getV(rasters, lonField, latField)
v1 = getV(rasters, lonEspla, latEspla)

y0, y1 = 35, 67

# Creating world-to-Minecraft Y transformation matrix

mY, hY = getMH(v0, y0, v1, y1)
v2y = make_affine_matrix([mY], [hY])

# Defining command text

prefix = "summon falling_block ~ ~1 ~ {Time:1,BlockState:{Name:redstone_block},Passengers:[\
{id:armor_stand,Health:0,Passengers:[\
{id:falling_block,Time:1,BlockState:{Name:activator_rail},Passengers:[\
{id:command_block_minecart,Command:'gamerule commandBlockOutput false'},\
{id:command_block_minecart,Command:'data merge block ~ ~-2 ~ {auto:0}'},"

template = "{{id:command_block_minecart,Command:'{0}'}}"

fill_command = "fill {0} {1} {2} {3} {4} {5} {6} {7} {8}"

clone_command = "clone {0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10}"

suffix = ",{id:command_block_minecart,Command:'setblock ~ ~1 ~ command_block{auto:1,Command:\"fill ~ ~ ~ ~ ~-2 ~ air\"}'},\
{id:command_block_minecart,Command:'kill @e[type=command_block_minecart,distance=..1]'}]}]}]}"

strings = {
    "prefix":        prefix,
    "template":      template,
    "fill_command":  fill_command,
    "clone_command": clone_command,
    "suffix":        suffix
}

# Command Generation

In [6]:
xMap, zMap = 9, -2
yield_map(rasters, strings, m2c, c2m, v2y, xMap, zMap)

Press Enter to store the next command in your clipboard... 


Batch 1 of 4


Press Enter to store the next command in your clipboard... 


Batch 2 of 4


Press Enter to store the next command in your clipboard... 


Batch 3 of 4


Press Enter to store the next command in your clipboard... 


Batch 4 of 4


# New features

In [ ]:
def load_ranges():
    try:
        with open("ranges.json", "r") as f:
            ranges = json.load(f)
    except FileNotFoundError:
        ranges = dict()
        
    return ranges

def reset_ranges():
    ranges = dict()
    
    with open("ranges.json", "w") as f:
        json.dump(ranges, f)
    
def save_range(pos, cmds, Y):
    ranges = load_ranges()
    x, z = pos
    x, z = str(x), str(z)
    y0, y1 = Y
    
    if ranges.get(x) is None:
        ranges[x] = dict()
    
    ranges[x][z] = {
        "start": int(y0),
        "end": int(y1),
        "cmds": cmds
    }
    
    with open("ranges.json", "w") as f:
        json.dump(ranges, f)

def mapTopLeft(mX, mZ):
    return mX * 128 - 64, mZ * 128 - 64

def package_commands(cmds, strings):
    max_chars = 32500
    prefix = strings["prefix"]
    suffix = strings["suffix"]
    
    def_len = len(prefix) + len(suffix)
    avail_space = max_chars - def_len
    lens = [len(cmd) + 1 for cmd in cmds]
    
    idx = 0
    batches = []
    
    while lens:
        idx = (np.cumsum(lens) <= avail_space).argmin(0)
        
        if idx == 0:
            idx = len(cmds)
            
        string = prefix + ",".join(cmds[:idx]) + suffix
        batches.append(string)
        lens = lens[idx:]
        cmds = cmds[idx:]
        
    for i, b in enumerate(batches):
        input("Press Enter to store the next command in your clipboard...")
        pc.copy(b)
        print(f"Batch {i + 1} of {len(batches)}")
        
def wipe_area(strings, pos, blocks, Y=None):
    x, z = pos
    
    block_map = {
        "d": "diamond_block",
        "g": "gold_block"
    }
    
    to_wipe = [v for k, v in block_map.items() if k in blocks]
        
    if Y is None:
        print("A")
        try:
            ranges = load_ranges()
            y0, y1 = tuple(ranges[str(x)][str(z)].values())[:2]
        except KeyError:
            print("The area (" + str(x) + ", " + str(z) + ") has not been seen yet.")
            return
    else:
        print("B")
        y0, y1 = Y

    x0, z0 = mapTopLeft(x, z)
    cmds = [gen_fill(strings, (127, 127, x0, z0), Y=y, block0="air", mode="replace", block1=block) for y in range(y0, y1) for block in to_wipe]
    package_commands(cmds, strings)

In [ ]:
# Turns a Minecraft heightmap into a set of individual commands yielded as batches
def cubify(arr, strings, shift=(0, 0)):
    multilayer, Y0, depth = thicken(arr.T)
    xShift, zShift = shift
    cmds = []
    
    for y in range(depth):
        layer = multilayer[..., y]
        sublayers = split_layer(layer)
        edges = []
        
        for (sub_layer, i, j) in sublayers:
            edges += tesselate_layer(sub_layer, i, j, xShift, zShift)
            
        cmds += [gen_fill(strings, elem, Y0 + y, "diamond_block", mode="keep") for elem in edges]
    
    package_commands(cmds, strings)
    
    return cmds, (Y0, Y0 + depth)

In [ ]:
def yield_map(rasters, strings, m2c, c2m, v2y, xMap, zMap, overwrite=False):
    xS, zS = mapTopLeft(xMap, zMap)
    arr = gen_heightmap(rasters, m2c, c2m, v2y, xS, zS)
    cmds, Y = cubify(arr, strings, shift=(xS, zS))
    save_range((xMap, zMap), cmds, Y)

In [ ]:
from helpers import gen_fill
import pyperclip as pc
import numpy as np
import json
from matplotlib import pyplot as plt

In [ ]:
wipe_area(strings, (1, 2), "dg")

In [ ]:
ranges = {
    np.int32(4) : 2
}
    
with open("temp.json", "w") as f:
    json.dump(ranges, f)

In [ ]:
np.int32(4)